### ![](../../img/installation-ico.png) Installation 

In [ ]:
pip install langchain langchain-community duckduckgo-search

### ![](../../img/package-ico.png) Imports

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

import os
import google.auth
from langchain_google_vertexai import ChatVertexAI, VertexAI

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

c:\Users\cleme\Workspace\sfeir-school-langchain\.venvSchool\Lib\site-packages\google_crc32c\__init__.py:29: RuntimeWarning: As the c extension couldn't be imported, `google-crc32c` is using a pure python implementation that is significantly slower. If possible, please configure a c build environment and compile the extension
  warnings.warn(_SLOW_CRC32C_WARNING, RuntimeWarning)


### ![](../../img/parametrage-ico.png) Paramétrages des outils

In [2]:
# Instantiation de l'outil de recherche

searchTool = DuckDuckGoSearchRun()

In [ ]:
# Informations sur l'outil

print(searchTool.name)
print(searchTool.description)
print(searchTool.args)

In [3]:
# Création d'un outil custom

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### ![](../../img/jouer-ico.png) Exécution des outils seuls

In [ ]:
# Utilisation de l'outil sans agent

searchTool.invoke("Who is Backack Obama ?")

In [ ]:
multiply.invoke({"a": 2, "b": 3})

### ![](../../img/parametrage-ico.png) Paramétrages de l'agent

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../settings/credentials.json"

google.auth.default()

(<google.oauth2.service_account.Credentials at 0x1f0411f0a40>,
 'fit-symbol-433212-u5')

In [5]:
# Création du prompt pour l'agent
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
    ]
)

In [6]:
llm = ChatVertexAI(model_name="gemini-1.5-pro")
# Définition de la liste des outils

tools = [multiply, searchTool]

# Initialisation de l'agent à partir du LLM, des outils et du prompt
agent = create_tool_calling_agent(llm, tools, prompt)

In [8]:
# Création du contexte d'exécution de l'agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
# Exécution utilisant l'outil de recherche

agent_executor.invoke({"input": "Who is Backack Obama ?"})



> Entering new AgentExecutor chain...


Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised InternalServerError: 500 Internal error encountered..



Invoking: `duckduckgo_search` with `{'query': 'Who is Barack Obama?'}`


Barack Obama (born August 4, 1961, Honolulu, Hawaii, U.S.) is the 44th president of the United States (2009-17) and the first African American to hold the office. Before winning the presidency, Obama represented Illinois in the U.S. Senate (2005-08). He was the third African American to be elected to that body since the end of ... They're meant to be old friends, so where is Barack Obama when the US vice-president needs him? Alex Hannaford talks to those who know them and uncovers a personal and political relationship ... It was in this highly polarized environment that Obama and the Democrats attempted to enact health care insurance reform. Barack Obama - 44th President, Political Career, Legacy: In 1996 he was elected to the Illinois Senate, where, most notably, he helped pass legislation that tightened campaign finance regulations, expanded health care to ... An earlier version of this article referred incorre

{'input': 'Who is Backack Obama ?',
 'output': "Barack Obama, born on August 4, 1961, in Honolulu, Hawaii, served as the 44th president of the United States from 2009 to 2017. He made history as the first African American to hold this position. Prior to his presidency, Obama represented Illinois in the U.S. Senate from 2005 to 2008, becoming the third African American elected to that body since the end of Reconstruction. \n\nObama's presidency was marked by significant events, including the passage of the Affordable Care Act, which aimed to expand health insurance coverage. He also navigated the country through the aftermath of the 2008 financial crisis. \n\nBeyond his political career, Obama is known for his close friendship with former Vice President Joe Biden. \n"}

In [ ]:
# Exécution utilisant l'outil de multiplication

agent_executor.invoke({"input": "What does 4 times 7 equal ?"})